In [1]:
import torch
from det3d.torchie import Config

import pickle

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
FINDCENTER_GEN_ONNX = True
POOLFORMER_GEN_ONNX = False

In [2]:
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu");

Deformable Convolution not built!
Deformable Convolution not built!
Use HM Bias:  -2.19


/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

In [3]:
from torch import nn

class CenterFinder(nn.Module):
    def __init__(self, model):
        super(CenterFinder, self).__init__()
        self.model = model

    def forward(self, x):
        # x = self.model.neck(x)
        # preds = self.model.bbox_head(x)
        # return preds
        return self.model.neck.find_centers(x) # pos_features, out_dict_list, ct_feat 
    
centerFinder = CenterFinder(model)
centerFinder.cuda()
centerFinder.eval();

In [4]:
import pickle
# with open("/workspace/centerformer/work_dirs/partition/sample_data/x.pkl", 'wb') as handle:
#     pickle.dump(x, handle)
# with open("/workspace/centerformer/work_dirs/partition/sample_data/example.pkl", 'wb') as handle:
#     pickle.dump(example, handle)

pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
# with open(pickle_dir + "example.pkl", 'rb') as handle:
#     example = pickle.load(handle)

In [5]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_orders, out_masks = centerFinder(x)
    
with open(pickle_dir + "findcenter_output1.pkl", 'rb') as handle:
    output1 = pickle.load(handle)
with open(pickle_dir + "findcenter_output2.pkl", 'rb') as handle:
    output2 = pickle.load(handle)
with open(pickle_dir + "findcenter_output3.pkl", 'rb') as handle:
    output3 = pickle.load(handle)
    
print((output1 == ct_feat).all())
print((output2 == center_pos_embedding).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


/root/anaconda3/envs/centerformer/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [6]:
import onnx

if FINDCENTER_GEN_ONNX:
    model_name = "findCenter.onnx"
    dummy_input=torch.randn(x.shape).cuda()
    torch.onnx.export(centerFinder, dummy_input, model_name,
                input_names=['input_tensor'], 
                output_names=['ct_feat', 'center_pos_embedding', 'out_scores', 'out_labels', 'out_orders', 'out_masks'],
                export_params=True, opset_version=11)
    
    # !export POLYGRAPHY_AUTOINSTALL_DEPS=1
    !polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx

    onnx.checker.check_model(onnx.load("findCenter_folded.onnx"))
    print("gen findCenter.onnx success!")
else:
    print("pass")

/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:932: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scores = scores.detach().cpu().numpy()
/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:934: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  order = torch.from_numpy(order).to(labels.device)
/data/centerformer/det3d/models/necks/rpn_transformer_multitask.py:935: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if yo

[I] RUNNING | Command: /root/anaconda3/envs/centerformer/bin/polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx
[I] Loading model: /data/centerformer/work_dirs/partition/onnx/findCenter.onnx
[I] Original Model:
    Name: torch-jit-export | ONNX Opset: 11
    
    ---- 1 Graph Input(s) ----
    {input_tensor [dtype=float32, shape=(1, 256, 180, 180)]}
    
    ---- 6 Graph Output(s) ----
    {ct_feat [dtype=float32, shape=(1, 3000, 256)],
     center_pos_embedding [dtype=float32, shape=(1, 3000, 256)],
     out_scores [dtype=float32, shape=(6, 1, 500)],
     out_labels [dtype=int64, shape=(6, 1, 500)],
     out_orders [dtype=int64, shape=(6, 1, 500)],
     out_masks [dtype=bool, shape=(6, 1, 500)]}
    
    ---- 132 Initializer(s) ----
    
    ---- 623 Node(s) ----
    
[I] Folding Constants | Pass 1
2023-12-13 17:02:04.587154121 [W:onnxruntime:, unsqueeze_elimination.cc:20 Apply] UnsqueezeElimination cannot remove node Unsqueeze_580
2023-12-13 

In [7]:
dummy_input.shape

torch.Size([1, 256, 180, 180])

In [8]:
with torch.no_grad():
    ct_feat = centerFinder.model.neck.poolformer_forward(ct_feat, center_pos_embedding)
            
out_dict_list = []
num_tasks = len(centerFinder.model.neck.tasks)
obj_num = centerFinder.model.neck.obj_num
for idx in range(num_tasks):
    out_dict = {}
    out_dict.update(
        {
            "scores": out_scores[idx],
            "labels": out_labels[idx],
            "order": out_orders[idx],
            "mask": out_masks[idx],
            "ct_feat": ct_feat[:, :, idx * obj_num : (idx+1) * obj_num],
        }
    )
    out_dict_list.append(out_dict)

In [9]:
with open(pickle_dir + "neck_output.pkl", 'rb') as handle:
    output_poolformer = pickle.load(handle)

In [10]:
out_dict_list[0].keys()

dict_keys(['scores', 'labels', 'order', 'mask', 'ct_feat'])

In [11]:
for i in range(len(out_dict_list)):
    print(i)
    print('scores', (output_poolformer[i]['scores']==out_dict_list[i]['scores']).all())
    print('labels', (output_poolformer[i]['labels']==out_dict_list[i]['labels']).all())
    print('orders', (output_poolformer[i]['order']==out_dict_list[i]['order']).all())
    print('masks', (output_poolformer[i]['mask']==out_dict_list[i]['mask']).all())
    print('ct_feat', (output_poolformer[i]['ct_feat']==out_dict_list[i]['ct_feat']).all())

0
scores tensor(True, device='cuda:0')
labels tensor(True, device='cuda:0')
orders tensor(True, device='cuda:0')
masks tensor(False, device='cuda:0')
ct_feat tensor(True, device='cuda:0')
1
scores tensor(True, device='cuda:0')
labels tensor(True, device='cuda:0')
orders tensor(True, device='cuda:0')
masks tensor(False, device='cuda:0')
ct_feat tensor(True, device='cuda:0')
2
scores tensor(True, device='cuda:0')
labels tensor(True, device='cuda:0')
orders tensor(True, device='cuda:0')
masks tensor(False, device='cuda:0')
ct_feat tensor(True, device='cuda:0')
3
scores tensor(True, device='cuda:0')
labels tensor(True, device='cuda:0')
orders tensor(True, device='cuda:0')
masks tensor(False, device='cuda:0')
ct_feat tensor(True, device='cuda:0')
4
scores tensor(True, device='cuda:0')
labels tensor(True, device='cuda:0')
orders tensor(True, device='cuda:0')
masks tensor(False, device='cuda:0')
ct_feat tensor(True, device='cuda:0')
5
scores tensor(True, device='cuda:0')
labels tensor(True, d

In [12]:
(output_poolformer[i]['mask']==out_dict_list[i]['mask'])

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, F